## Description:

This notebook aims to use the data within the `IAQF_DATA.csv` file to create the master dataframe for the competition. 

In [178]:
import pandas as pd

all_df = pd.read_csv('./IAQF_DATA.csv')

In [179]:
all_df.head()

,Dates,SP500,vol,Dates_rfrate,rf_rate
0,1/18/05,1195.98,10.49,1/18/05,2.39
1,1/19/05,1184.63,10.64,1/19/05,2.37
2,1/20/05,1175.41,10.73,1/20/05,2.35
3,1/21/05,1167.87,10.71,1/21/05,2.36
4,1/24/05,1163.75,10.73,1/24/05,2.37


In [180]:
rates_df = pd.DataFrame()
rates_df['Dates'] = all_df['Dates_rfrate']
rates_df['rf_rate'] = all_df['rf_rate']

del all_df['Dates_rfrate']
del all_df['rf_rate']

In [181]:
all_df.head()

,Dates,SP500,vol
0,1/18/05,1195.98,10.49
1,1/19/05,1184.63,10.64
2,1/20/05,1175.41,10.73
3,1/21/05,1167.87,10.71
4,1/24/05,1163.75,10.73


In [182]:
rates_df.head()

,Dates,rf_rate
0,1/18/05,2.39
1,1/19/05,2.37
2,1/20/05,2.35
3,1/21/05,2.36
4,1/24/05,2.37


In [183]:
master = pd.merge(all_df, rates_df, how='inner', on=['Dates'])
master.head()

,Dates,SP500,vol,rf_rate
0,1/18/05,1195.98,10.49,2.39
1,1/19/05,1184.63,10.64,2.37
2,1/20/05,1175.41,10.73,2.35
3,1/21/05,1167.87,10.71,2.36
4,1/24/05,1163.75,10.73,2.37


In [186]:
from scipy.stats import norm
from math import log, sqrt, exp

def d_plus( x, K, tau, sigma, rfr ):
    
    tau /= 360
    rfr /= 100
    sigma /= 100
    
    return ( (log(x/K) + ((rfr + 0.5*(sigma**2))*tau))/(sigma*sqrt(tau)) )

def d_minus( x, K, tau, sigma, rfr ):
    
    tau /= 360
    rfr /= 100
    sigma /= 100
    
    return ( d_plus( x, K, tau, sigma, rfr ) - sigma*sqrt(tau) )


def bs_call_price( x, K, tau, sigma, rfr ):
    
    tau /= 360
    rfr /= 100
    sigma /= 100
    d_1 = d_plus( x, K, tau, sigma, rfr )
    d_2 = d_minus( x, K, tau, sigma, rfr )
    
    print()
    
    return ( x*norm.cdf(d_1) - K*exp(-1.0*rfr*tau)*norm.cdf(d_2) )

def bs_put_price( x, K, tau, sigma, rfr ):
    
    tau /= 360
    rfr /= 100
    sigma /= 100
    d_1 = d_plus( x, K, tau, sigma, rfr )
    d_2 = d_minus( x, K, tau, sigma, rfr )
    
    print("x: ", x)
    print("K: ", K)
    
    return ( K*exp(-1.0*rfr*tau)*norm.cdf(-1.0*d_2) - x*norm.cdf(-1.0*d_1) )

def put_from_pcp( x, K, tau, sigma, rfr ):
    
    tau /= 360
    rfr /= 100
    sigma /= 100
    d_1 = d_plus( x, K, tau, sigma, rfr )
    d_2 = d_minus( x, K, tau, sigma, rfr )
    
    return ( bs_call_price(x, K, tau, sigma, rfr) - x + K*exp(-1.0*rfr*tau) )
    
    

In [187]:
import numpy as np

master['bs_call'] = np.vectorize(bs_call_price)(master.SP500, master.SP500, 90, master.vol, master.rf_rate)
#master['bs_put'] = np.vectorize(bs_put_price)(master.SP500, master.SP500, 90, master.vol, master.rf_rate)
#master['bs_put_pcp'] = np.vectorize(put_from_pcp)(master.SP500, master.SP500, 90, master.vol, master.rf_rate)
master.head()

,Dates,SP500,vol,rf_rate,bs_call
0,1/18/05,1195.98,10.49,2.39,6.296614
1,1/19/05,1184.63,10.64,2.37,6.147602
2,1/20/05,1175.41,10.73,2.35,6.026999
3,1/21/05,1167.87,10.71,2.36,6.018438
4,1/24/05,1163.75,10.73,2.37,6.017750
